In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv(r'../input/ffdata/Reviews.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['helpful_perc']=np.where(df['HelpfulnessDenominator']>0,df['HelpfulnessNumerator']/df['HelpfulnessDenominator'],-1)

In [ ]:
## defining bins

df['upvote_perc']=pd.cut(df['helpful_perc'],bins=[-1,0,0.2,0.4,0.6,0.8,1],labels=['Empty','0-20%','20-40%','40-60%','60-80%','80-100%'])

In [ ]:
df.head()

#### Analyzing Upvotes based on scores.

In [ ]:
df.groupby(['Score','upvote_perc']).agg('count')

In [ ]:
df_s= df.groupby(['Score','upvote_perc']).agg({'Id':'count'}).reset_index()

In [ ]:
df_s

In [ ]:
pivot=df_s.pivot(index='upvote_perc',columns='Score')
pivot

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(pivot,annot=True, cmap='YlGnBu')

#### We can notice that most of the reviews are 5 star

In [ ]:
df['Score'].unique()

In [ ]:
## Assuming neutral reviews to be 3, therefore filtering them

df2 = df[df['Score']!=3]

In [ ]:
df2['Score'].unique()

#### Defining X and Y

In [ ]:
X = df2['Text']

In [ ]:
## before defining y we need to map the reviews with 0 & 1, where reviews with 1&2 represent 0 and 4&5 represent 1

y_dict = {1:0, 2:0, 4:1, 5:1}

y = df2['Score'].map(y_dict)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
c = CountVectorizer(stop_words='english')

In [ ]:
X_c = c.fit_transform(X)

In [ ]:
X_c.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X_c,y)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log= LogisticRegression()

In [ ]:
ml = log.fit(X_train,y_train)

In [ ]:
ml.score(X_test,y_test)

### Top 20 positive & negative words

In [ ]:
## getting feature names

w= c.get_feature_names()

In [ ]:
coef= ml.coef_.tolist()[0]

In [ ]:
coef_df= pd.DataFrame({"Word":w,"coeffcient":coef})
coef_df.head()

In [ ]:
coef_df= coef_df.sort_values(['coeffcient','Word'],ascending=False)

In [ ]:
## Top 20 positive words

coef_df.head(20)

In [ ]:
coef_df.tail(20)

### Predictions

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
def predict(X,y,nlp_model,ml_model):
    X_c=nlp_model.fit_transform(X)
    X_train,X_test,y_train,y_test= train_test_split(X_c,y)
    ml = ml_model.fit(X_train,y_train)
    predictions= ml.predict(X_test)
    cm= confusion_matrix(predictions, y_test)
    print(cm)
    acc= accuracy_score(predictions, y_test)
    print(acc)

In [ ]:
c=CountVectorizer()
lr=LogisticRegression()

In [ ]:
predict(X,y,c,lr)

### Upvote Prediction

In [ ]:
### Per heatmap high number of users have left five star reviews, therefore we'll filter these review for analysis

data= df[df['Score']==5]
data.head()

In [ ]:
data['upvote_perc'].unique()

In [ ]:
## removing nuetral votes

data2= data[data['upvote_perc'].isin(['80-100%','60-80%','20-40%', '0-20%'])]

In [ ]:
## setting up independent data

X= data2['Text']

In [ ]:
y_dict= {'80-100%':1,'60-80%':1,'20-40%':0, '0-20%':0}

y= data2['upvote_perc'].map(y_dict)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf= TfidfVectorizer(stop_words='english')

In [ ]:
x_c= tf.fit_transform(X)

### Handling Imbalance Data

In [ ]:
!pip install -U tensorflow==2.0.0
import tensorflow as tf
tf.__version__

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
os= RandomOverSampler()

In [ ]:
X_train_res,y_train_res= os.fit_resample(x_c, y)

In [ ]:
X_train_res.shape

In [ ]:
y_train_res.shape

In [ ]:
from collections import Counter

In [ ]:
print('Original datset shape {}'.format(Counter(y)))
print('Resampled datset shape {}'.format(Counter(y_train_res)))

### Cross-validation using Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
log_class= LogisticRegression()

In [ ]:
grid= {'C':10.0**np.arange(-2,3), 'penalty':['l1', 'l2']}

In [ ]:
clf= GridSearchCV(estimator= log_class, param_grid= grid, cv=5, n_jobs= -1, scoring='f1_macro')

In [ ]:
clf.fit(X_train_res, y_train_res)

In [ ]:
## train test split

X_train,X_test,y_train,y_test= train_test_split(x_c, y)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
## Confusion matrix

confusion_matrix(y_test, pred)

In [ ]:
## Accuracy score

accuracy_score(y_test, pred)